<a href="https://colab.research.google.com/github/eduardobbastos/colabs/blob/main/transcri_o_audio_video_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎧📝 Transcrição automática de áudio ou vídeo no Google Colab

Bem-vindo! Este notebook permite **extrair a transcrição de qualquer áudio ou vídeo**, seja por upload de arquivo ou por URL (YouTube, Vimeo, Spotify, etc.), usando o modelo Whisper da OpenAI.

## **Como usar este notebook**

1. **Execute a célula de instalação de pacotes:**  
   Clique no botão de “play” (▶️) na primeira célula de código para instalar as dependências necessárias.  
   _Aguarde até aparecer “Successfully installed...” antes de seguir para as próximas células._

2. **Rode a célula principal do código:**  
   - Escolha o que você deseja transcrever:
     - **1** para vídeo (áudio será extraído automaticamente)
     - **2** para áudio (direto do arquivo ou da URL)
   - Depois, escolha entre fazer upload do arquivo ou informar a URL.
     - **Upload:** selecione o arquivo do seu computador.
     - **URL:** cole o link do vídeo ou do áudio (ex: do YouTube, Vimeo, ou link direto de áudio .mp3).
   - O notebook irá baixar/processar seu arquivo, extrair o áudio se for vídeo e gerar a transcrição.

3. **Aguarde a transcrição:**  
   O tempo de processamento depende do tamanho do arquivo e do modelo usado (por padrão, o modelo é `medium`, com boa fidelidade).

4. **Baixe o arquivo de transcrição:**  
   Após o processamento, um arquivo `.txt` com o texto transcrito será gerado e disponibilizado para download.

---

> **Dica:**  
> Para máxima qualidade, utilize arquivos com boa clareza de áudio.  
> Para arquivos longos ou para máxima precisão, você pode alterar o modelo para `"large"` no código, mas pode demorar mais e consumir mais memória.

---

Pronto! Agora, basta seguir as instruções no notebook e processar seus áudios e vídeos para obter a transcrição em texto, pronta para ser usada em qualquer aplicação de IA ou automação.



In [ ]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg
!pip install moviepy yt-dlp

In [ ]:
import yt_dlp
import os
from moviepy import VideoFileClip
from google.colab import files
import whisper
from datetime import datetime

# Função para gerar timestamp
def get_timestamp():
    return datetime.now().strftime('%Y%m%d_%H%M%S')

def download_video(url, output_path_base='downloaded_video'):
    timestamp = get_timestamp()
    output_path = f"{output_path_base}_{timestamp}"
    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',
        'outtmpl': f'{output_path}.%(ext)s',
        'merge_output_format': 'mp4'
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        if 'requested_downloads' in info:
            ext = info['requested_downloads'][0]['ext']
        else:
            ext = info['ext'] if 'ext' in info else 'mp4'
    return f"{output_path}.{ext}"

def download_audio(url, output_path_base='downloaded_audio'):
    timestamp = get_timestamp()
    output_path = f"{output_path_base}_{timestamp}"
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': f'{output_path}.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'prefer_ffmpeg': True
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        if 'requested_downloads' in info:
            ext = info['requested_downloads'][0]['ext']
        else:
            ext = info['ext'] if 'ext' in info else 'mp3'
    return f"{output_path}.mp3"

# --- Interface de escolha do usuário ---
print('O que você deseja transcrever?')
print('1 - Vídeo (o áudio será extraído)')
print('2 - Áudio (direto do arquivo ou URL)')
task = input('Digite 1 ou 2: ').strip()

if task == '1':
    print('Transcrição de VÍDEO selecionada.')
    print('Escolha o método:\n1 - Upload do arquivo de vídeo\n2 - URL de vídeo')
    method = input('Digite 1 ou 2: ').strip()
    if method == '1':
        print('Faça upload do seu vídeo...')
        uploaded = files.upload()
        video_path_base = list(uploaded.keys())[0]
        file_ext = os.path.splitext(video_path_base)[1]
        timestamp = get_timestamp()
        video_path = f"{os.path.splitext(video_path_base)[0]}_{timestamp}{file_ext}"
        os.rename(video_path_base, video_path)
    elif method == '2':
        url = input('Cole a URL do vídeo: ').strip()
        print('Baixando vídeo...')
        video_path = download_video(url)
        print('Download concluído:', video_path)
    else:
        print('Opção inválida.')
        raise SystemExit
    # Extrair áudio do vídeo
    print('Extraindo áudio do vídeo...')
    audio_path = os.path.splitext(video_path)[0] + '_audio.mp3'
    clip = VideoFileClip(video_path)
    clip.audio.write_audiofile(audio_path, codec='mp3', bitrate='320k')
    clip.close()
elif task == '2':
    print('Transcrição de ÁUDIO selecionada.')
    print('Escolha o método:\n1 - Upload do arquivo de áudio\n2 - URL de áudio')
    method = input('Digite 1 ou 2: ').strip()
    if method == '1':
        print('Faça upload do seu áudio...')
        uploaded = files.upload()
        audio_path_base = list(uploaded.keys())[0]
        file_ext = os.path.splitext(audio_path_base)[1]
        timestamp = get_timestamp()
        audio_path = f"{os.path.splitext(audio_path_base)[0]}_{timestamp}{file_ext}"
        os.rename(audio_path_base, audio_path)
    elif method == '2':
        url = input('Cole a URL do áudio: ').strip()
        print('Baixando áudio...')
        audio_path = download_audio(url)
        print('Download concluído:', audio_path)
    else:
        print('Opção inválida.')
        raise SystemExit
else:
    print('Opção inválida.')
    raise SystemExit

# --- Transcrição automática ---
print('Carregando modelo Whisper (medium)...')
model = whisper.load_model("medium")  # Altere para "large" se quiser máxima fidelidade
print('Transcrevendo, aguarde...')
result = model.transcribe(audio_path, fp16=False, language="pt")

# --- Salvar e baixar transcrição ---
txt_path = os.path.splitext(audio_path)[0] + '_transcription.txt'
with open(txt_path, "w", encoding="utf-8") as f:
    f.write(result["text"])
print('Transcrição salva em:', txt_path)
files.download(txt_path)
print('Processo finalizado com sucesso!')
